In [ ]:
#Solve ToxicDetection through feature engineer into logistic regression
# Reference this tutorial for the logistic regression: 
##  https://www.kaggle.com/negation/pytorch-logistic-regression-tutorial
%load_ext autoreload
%autoreload 2


In [ ]:
#Input Data needs to be in array of size N x (2)
import utils 
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



In [ ]:
pd.set_option('display.max_colwidth', -1)
root_dir = os.path.abspath('.')
data_dir = os.path.join(root_dir, 'dataset')
train = pd.read_csv(os.path.join(data_dir,'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))


train_x, train_y = utils.featurize(train)
#test_x, test_y = utils.featurize(test)



In [ ]:


class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes=2):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        print("Input coming in as", x.shape)
        return self.linear(x)
        

In [ ]:
#train on data_train with several epochs
epochs = 20 #Can also be changed. The higher this value, the more overfit. 
feature_dims = 21
batchSize = 64
classes = 6 #Binaray for toxic, or not
model = LogisticRegression(feature_dims, num_classes=classes)
model = model.cuda() #COMMENT OUT TO RUN ON GPU
#Todo: set loss criterion and optimizer
#Run loop of number of epoch, forward and backpass, loss then into optimizer
lossFn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

#Training
for epoch in range(epochs):
    sampleIdx = np.random.choice(np.arange(len(train_y)), batchSize, replace=False)
    x_s = torch.from_numpy(train_x[sampleIdx]).float().to(device)
    y_s = torch.from_numpy(train_y[sampleIdx]).to(device)
    #for i in range(batchSize): 
        #print(x_s[i].shape, y_s[i].shape)
    optimizer.zero_grad()
    scores = model(x_s)
    print(scores.shape, y_s.shape)
    loss = lossFn(scores, y_s)
    loss.backward()
    optimizer.step()
        


    


In [ ]:
#TODO:
#Optimize hyperparameters like batchsize, or learning rate or
#Try to add dropout, batch normalization to get highest val acc

In [ ]:
#TODO: 
#Now try test set, hopefully for accuracy as good as validation set